<a href="https://colab.research.google.com/github/NourKamaly/TheArtInOurWorlds-NASA-Space-Apps/blob/main/integrated_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install diffusers transformers scipy mediapy deep_translator gtts langdetect pytextrank sentence-transformers sklearn

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [69]:
from deep_translator import GoogleTranslator
def translateSentence(sentence,lang='en'):
  return GoogleTranslator('auto',lang).translate(sentence)

In [4]:
from gtts import gTTS
from langdetect import detect, DetectorFactory
def TextToSpeech(text):
  DetectorFactory.seed = 0
  language=detect(text)
  print(language)
  try:
    ttsObj = gTTS(text=text, lang=language, slow=False)
  except:
     ttsObj = gTTS(text=text, lang='ar', slow=False)
  ttsObj.save(f"test.mp3")

In [41]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import sent_tokenize

from sentence_transformers import SentenceTransformer
import time


In [64]:
def process_bert_similarity(base_document,documents):
  # This will download and load the pretrained model offered by UKPLab.
  model = SentenceTransformer('bert-base-nli-mean-tokens')

	# Although it is not explicitly stated in the official document of sentence transformer, the original BERT is meant for a shorter sentence. We will feed the model by sentences instead of the whole documents.
  sentences = sent_tokenize(base_document)
  base_embeddings_sentences = model.encode(sentences)
  base_embeddings = np.mean(np.array(base_embeddings_sentences), axis=0)
  
  vectors = []
  for i, document in enumerate(documents):
    sentences = sent_tokenize(document)
    embeddings_sentences = model.encode(sentences)
    embeddings = np.mean(np.array(embeddings_sentences), axis=0)
    vectors.append(embeddings)
    #print("making vector at index:", i)
  scores = cosine_similarity([base_embeddings], vectors).flatten()

  highest_score = 0
  highest_score_index = 0
  score_dict={}
  for i, score in enumerate(scores):
        score_dict[score]=i
  score_dict=sorted(score_dict.items(), key=lambda x: x[0])
  return  score_dict[-10:]
  most_similar_document = documents[highest_score_index]
  print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")
  print(f"Most similar document: {most_similar_document} \n\n\nThe Score: {highest_score}")

In [11]:
import pandas as pd
data = pd.read_csv('APOD.csv')

In [12]:
data.columns

Index(['Unnamed: 0', 'date', 'explanation', 'hdurl', 'media_type',
       'service_version', 'title', 'url', 'copyright'],
      dtype='object')

In [14]:
docs=data.explanation.tolist()

In [82]:
text='Ich möchte auf dem Mars leben'
DetectorFactory.seed = 0
language=detect(text)
eng_text=translateSentence(text)
scores_dict=process_bert_similarity(eng_text,docs[:60])


In [83]:
language=detect(text)

In [84]:
TextToSpeech(translateSentence(docs[scores_dict[-1][1]],language))

de


In [77]:
!pip -q install wget

In [88]:
import wget
files=[]
for i in scores_dict:
  files.append(wget.download(data.url[i[1]]))

In [102]:
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline

In [ ]:
!pip install simpletransformers


In [ ]:
!pip install sentencepiece


In [105]:
model_name = "google/pegasus-xsum"
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)


In [108]:
# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Create tokens
tokens = pegasus_tokenizer(text, truncation=True, padding="longest", return_tensors="pt")



Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [109]:
encoded_summary = pegasus_model.generate(**tokens)

# Decode summarized text
decoded_summary = pegasus_tokenizer.decode(
      encoded_summary[0],
      skip_special_tokens=True
)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [110]:
decoded_summary


'Nasa is planning to send a team of scientists to Mars to explore the Red Planet.'

In [111]:
summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)

In [115]:
summary = summarizer(docs[1], min_length=30, max_length=150)
summary[0]["summary_text"]

Your max_length is set to 150, but you input_length is only 148. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)


'Each day we feature a photograph taken from the skies by an amateur astronomer. Astronomy Picture of the Day is brought to you by Robert Nemiroff and Jerry Bonnell .'